# INF582 AXA Challenge

## Initialisation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sb
import itertools
import random

pd.set_option('display.max_columns', 500)

## Load data

In [ ]:
# Load data
training_data = pd.read_csv("data/train_2011_2012.csv", sep=';', nrows=10000,
                            na_values=['NaN', 'A Définir', 'A DEFINIR', '9999-12-31 00:00:00.000'])
training_data.head()

## Clean data

Boris: I think we should group by ASS_ASSIGNMENT and DATE (it seems the data has been split by teams or whatever)

In [ ]:
# Remove columns containing only one value
for name in training_data.columns:
    count = training_data[name].unique().size
    if count <= 1:
        training_data.drop(name, 1, inplace=True)
        
training_data.head(n=10)

## View main statistics

In [ ]:
training_data.describe()

##  A simple predictor

TODO : Lets try building a tree-based boosting predictor with very few attributes just to see how it goes.
This predictor will only predict the number of calls received during a given time stamp (e.g. in a 30 minutes slot)

In [ ]:
#Select only basic attributes
training_data_simple = training_data[['DATE', 'ASS_ASSIGNMENT','DAY_WE_DS','WEEK_END','CSPL_RECEIVED_CALLS']]
#Aggregate the calls
grouped = training_data_simple.groupby(['DATE','ASS_ASSIGNMENT','DAY_WE_DS','WEEK_END'])
ag = grouped['CSPL_RECEIVED_CALLS'].aggregate(np.sum)
ag.head(n=100)